# 1. Load index

In [1]:
import genomics_data_index

print(genomics_data_index.__version__)

0.4.0


In [2]:
import genomics_data_index.api as gdi

db = gdi.GenomicsDataIndex.connect('cases/unset/index')
db

<GenomicsDataIndex(samples=22)>

# 2. Load metadata

In [3]:
import pandas as pd

metadata_df = pd.read_csv('metadata.tsv', sep='\t')
metadata_df['cluster'] = metadata_df['outbreak']
metadata_df.loc[metadata_df['cluster'] == 'outgroup', 'cluster'] = pd.NA
metadata_df

dataSetName       strain              outbreak  \
0                          0810PADBR-1        D5663  outgroup_0810PADBR-1   
1                          0810PADBR-1        D7321           0810PADBR-1   
2                          0810PADBR-1        D7331           0810PADBR-1   
3                          0810PADBR-1        D7319           0810PADBR-1   
4                          0810PADBR-1        D7323           0810PADBR-1   
..                                 ...          ...                   ...   
80  1203NYJAP-1 - Tuna Scrape Outbreak  CFSAN000970           1203NYJAP-1   
81  1203NYJAP-1 - Tuna Scrape Outbreak  CFSAN001112           1203NYJAP-1   
82  1203NYJAP-1 - Tuna Scrape Outbreak  CFSAN001115           1203NYJAP-1   
83  1203NYJAP-1 - Tuna Scrape Outbreak  CFSAN001118           1203NYJAP-1   
84  1203NYJAP-1 - Tuna Scrape Outbreak  CFSAN001140           1203NYJAP-1   

    SRArun_acc               cluster  
0   SRR1999649  outgroup_0810PADBR-1  
1   SRR3215135           0810PADBR-1  
2   SRR3214715           0810PADBR-1  
3   SRR3215210           0810PADBR-1  
4   SRR3215124           0810PADBR-1  
..         ...                   ...  
80   SRR498444           1203NYJAP-1  
81  SRR1258439           1203NYJAP-1  
82  SRR1258442           1203NYJAP-1  
83  SRR1258443           1203NYJAP-1  
84  SRR1258440           1203NYJAP-1  

[85 rows x 5 columns]

# 3. Kmer tree

In [4]:
q = db.samples_query().join(metadata_df, sample_names_column='strain')
q

<DataFrameSamplesQuery[selected=100% (22/22) samples, unknown=0% (0/22) samples]>

In [5]:
qkmer_31 = q.build_tree(kind='kmer', kmer_size=31)
qkmer_31

<KmerTreeSamplesQuery[selected=100% (22/22) samples, unknown=0% (0/22) samples]>

In [6]:
q_campy = qkmer_31.isa('0810PADBR-1', isa_column='dataSetName', kind='dataframe')
q_ecoli = qkmer_31.isa('1405WAEXK-1', isa_column='dataSetName', kind='dataframe')
q_listeria = qkmer_31.isa('1408MLGX6-3WGS', isa_column='dataSetName', kind='dataframe')
q_salmonella = qkmer_31.isa('1203NYJAP-1 - Tuna Scrape Outbreak', isa_column='dataSetName', kind='dataframe')

In [7]:
qkmer_31.tree_styler()\
 .highlight(q_campy, legend_label='Campy')\
 .highlight(q_ecoli, legend_label='Ecoli')\
 .highlight(q_listeria, legend_label='Listeria')\
 .highlight(q_salmonella, legend_label='Salmonella')\
 .render(w=300)

In [8]:
from genomics_data_index.api.query.impl.ClusterScorer import ClusterScorer

min_samples_cluster = 2

scorer = ClusterScorer(qkmer_31)
scores_organism = scorer.score_groupby(groupby_column='dataSetName', min_samples_count=min_samples_cluster)
scores_organism

Score  Sample Count
dataSetName                     
0810PADBR-1    1.0            22

In [9]:
scores_outbreak = scorer.score_groupby(groupby_column='cluster', min_samples_count=min_samples_cluster)
scores_outbreak

Score  Sample Count
cluster                                     
0810PADBR-1           1.000000            14
outgroup_0810PADBR-1  0.363636             8

# ML Tree

In [10]:
db.reference_names()

['GCA_001879185.2_ASM187918v2_genomic']

In [11]:
t = db.samples_query(reference_name='GCA_001879185.2_ASM187918v2_genomic', universe='mutations')\
      .set_outgroup('GCA_001879185.2_ASM187918v2_genomic')\
      .join(metadata_df, sample_names_column='strain')
t

<MutationTreeSamplesQuery[selected=100% (22/22) samples, unknown=0% (0/22) samples]>

In [18]:
from ete3 import NodeStyle

node_style = NodeStyle()
node_style['size'] = 3

t.tree_styler(node_style=node_style)\
 .highlight(t.isa('0810PADBR-1', isa_column='outbreak', kind='dataframe'), legend_label='0810PADBR-1')\
 .highlight(t.isa('outgroup_0810PADBR-1', isa_column='outbreak', kind='dataframe'), legend_label='outgroup_0810PADBR-1')\
 .render(w=300, ladderize=True)

In [13]:
scorer_ml = ClusterScorer(t)
scores_outbreak = scorer_ml.score_groupby(groupby_column='cluster', min_samples_count=min_samples_cluster)
scores_outbreak

Score  Sample Count
cluster                                     
0810PADBR-1           0.636364            14
outgroup_0810PADBR-1  1.000000             8

In [14]:
t.isa('0810PADBR-1', isa_column='outbreak', kind='dataframe').features_summary(selection='unique').sort_values('Count', ascending=False)

Sequence  Position Deletion Insertion   Type  \
Mutation                                                                   
CP069124.1:1190591:A:AC   CP069124.1   1190591        A        AC  INDEL   
CP069124.1:1376739:T:TG   CP069124.1   1376739        T        TG  INDEL   
CP069124.1:1364330:T:TG   CP069124.1   1364330        T        TG  INDEL   
CP069124.1:741372:A:AG    CP069124.1    741372        A        AG  INDEL   
CP069124.1:1369265:C:T    CP069124.1   1369265        C         T    SNP   
CP069124.1:807524:TC:T    CP069124.1    807524       TC         T  INDEL   
CP069124.1:1348343:A:AC   CP069124.1   1348343        A        AC  INDEL   
CP069124.1:1365317:T:TGG  CP069124.1   1365317        T       TGG  INDEL   

                          Count  Total     Percent          Annotation  \
Mutation                                                                 
CP069124.1:1190591:A:AC      14     14  100.000000  frameshift_variant   
CP069124.1:1376739:T:TG      11     14   78.571429  frameshift_variant   
CP069124.1:1364330:T:TG       9     14   64.285714  frameshift_variant   
CP069124.1:741372:A:AG        6     14   42.857143  frameshift_variant   
CP069124.1:1369265:C:T        1     14    7.142857    missense_variant   
CP069124.1:807524:TC:T        1     14    7.142857      intron_variant   
CP069124.1:1348343:A:AC       1     14    7.142857  frameshift_variant   
CP069124.1:1365317:T:TGG      1     14    7.142857  frameshift_variant   

                         Annotation_Impact     Gene_Name       Gene_ID  \
Mutation                                                                 
CP069124.1:1190591:A:AC               HIGH  A0X86_006255  A0X86_006255   
CP069124.1:1376739:T:TG               HIGH  A0X86_007190  A0X86_007190   
CP069124.1:1364330:T:TG               HIGH  A0X86_007130  A0X86_007130   
CP069124.1:741372:A:AG                HIGH          kdpB  A0X86_003980   
CP069124.1:1369265:C:T            MODERATE          legG  A0X86_007155   
CP069124.1:807524:TC:T            MODIFIER          hisA  A0X86_000005   
CP069124.1:1348343:A:AC               HIGH  A0X86_007050  A0X86_007050   
CP069124.1:1365317:T:TGG              HIGH  A0X86_007135  A0X86_007135   

                         Feature_Type Transcript_BioType            HGVS.c  \
Mutation                                                                     
CP069124.1:1190591:A:AC    transcript     protein_coding         c.306dupG   
CP069124.1:1376739:T:TG    transcript     protein_coding         c.134dupG   
CP069124.1:1364330:T:TG    transcript     protein_coding         c.673dupG   
CP069124.1:741372:A:AG     transcript     protein_coding         c.807dupG   
CP069124.1:1369265:C:T     transcript     protein_coding         c.1058C>T   
CP069124.1:807524:TC:T     transcript     protein_coding  c.350+807175delC   
CP069124.1:1348343:A:AC    transcript     protein_coding         c.585dupG   
CP069124.1:1365317:T:TGG   transcript     protein_coding    c.961_962dupGG   

                            HGVS.p  \
Mutation                             
CP069124.1:1190591:A:AC   p.Y103fs   
CP069124.1:1376739:T:TG    p.I46fs   
CP069124.1:1364330:T:TG   p.D225fs   
CP069124.1:741372:A:AG    p.L270fs   
CP069124.1:1369265:C:T     p.P353L   
CP069124.1:807524:TC:T        <NA>   
CP069124.1:1348343:A:AC   p.S196fs   
CP069124.1:1365317:T:TGG  p.S322fs   

                                                              ID_HGVS.c  \
Mutation                                                                  
CP069124.1:1190591:A:AC          hgvs:CP069124.1:A0X86_006255:c.306dupG   
CP069124.1:1376739:T:TG          hgvs:CP069124.1:A0X86_007190:c.134dupG   
CP069124.1:1364330:T:TG          hgvs:CP069124.1:A0X86_007130:c.673dupG   
CP069124.1:741372:A:AG           hgvs:CP069124.1:A0X86_003980:c.807dupG   
CP069124.1:1369265:C:T           hgvs:CP069124.1:A0X86_007155:c.1058C>T   
CP069124.1:807524:TC:T    hgvs:CP069124.1:A0X86_000005:c.350+807175delC   
CP069124.1:1348343:A:AC

In [15]:
df = t.isa('D7331').features_summary()
# df[df['Type'] != 'INDEL']
df

Sequence  Position Deletion Insertion   Type  \
Mutation                                                                  
CP069124.1:1299601:C:CT  CP069124.1   1299601        C        CT  INDEL   
CP069124.1:1160157:A:AT  CP069124.1   1160157        A        AT  INDEL   
CP069124.1:898050:C:CT   CP069124.1    898050        C        CT  INDEL   
CP069124.1:877318:T:TA   CP069124.1    877318        T        TA  INDEL   
CP069124.1:873531:C:CA   CP069124.1    873531        C        CA  INDEL   
...                             ...       ...      ...       ...    ...   
CP069124.1:1445467:C:CT  CP069124.1   1445467        C        CT  INDEL   
CP069124.1:1469012:A:AC  CP069124.1   1469012        A        AC  INDEL   
CP069124.1:467023:G:GA   CP069124.1    467023        G        GA  INDEL   
CP069124.1:642794:C:CA   CP069124.1    642794        C        CA  INDEL   
CP069124.1:58819:A:AT    CP069124.1     58819        A        AT  INDEL   

                         Count  Total  Percent          Annotation  \
Mutation                                                             
CP069124.1:1299601:C:CT      1      1    100.0  frameshift_variant   
CP069124.1:1160157:A:AT      1      1    100.0  frameshift_variant   
CP069124.1:898050:C:CT       1      1    100.0  frameshift_variant   
CP069124.1:877318:T:TA       1      1    100.0  frameshift_variant   
CP069124.1:873531:C:CA       1      1    100.0  frameshift_variant   
...                        ...    ...      ...                 ...   
CP069124.1:1445467:C:CT      1      1    100.0  frameshift_variant   
CP069124.1:1469012:A:AC      1      1    100.0  frameshift_variant   
CP069124.1:467023:G:GA       1      1    100.0                <NA>   
CP069124.1:642794:C:CA       1      1    100.0      intron_variant   
CP069124.1:58819:A:AT        1      1    100.0  frameshift_variant   

                        Annotation_Impact     Gene_Name       Gene_ID  \
Mutation                                                                
CP069124.1:1299601:C:CT              HIGH  A0X86_006805  A0X86_006805   
CP069124.1:1160157:A:AT              HIGH          ftsH  A0X86_006110   
CP069124.1:898050:C:CT               HIGH          metG  A0X86_004750   
CP069124.1:877318:T:TA               HIGH  A0X86_004640  A0X86_004640   
CP069124.1:873531:C:CA               HIGH  A0X86_004620  A0X86_004620   
...                                   ...           ...           ...   
CP069124.1:1445467:C:CT              HIGH          feoB  A0X86_007485   
CP069124.1:1469012:A:AC              HIGH  A0X86_007600  A0X86_007600   
CP069124.1:467023:G:GA               HIGH          serS  A0X86_002545   
CP069124.1:642794:C:CA           MODIFIER          hisA  A0X86_000005   
CP069124.1:58819:A:AT                HIGH          traF  A0X86_000340   

                        Feature_Type Transcript_BioType  \
Mutation                                                  
CP069124.1:1299601:C:CT   transcript     protein_coding   
CP069124.1:1160157:A:AT   transcript     protein_coding   
CP069124.1:898050:C:CT    transcript     protein_coding   
CP069124.1:877318:T:TA    transcript     protein_coding   
CP069124.1:873531:C:CA    transcript     protein_coding   
...                              ...                ...   
CP069124.1:1445467:C:CT   transcript     protein_coding   
CP069124.1:1469012:A:AC   transcript     protein_coding   
CP069124.1:467023:G:GA    transcript     protein_coding   
CP069124.1:642794:C:CA    transcript     protein_coding   
CP069124.1:58819:A:AT     transcript     protein_coding   

                                              HGVS.c    HGVS.p  \
Mutation                                                         
CP069124.1:1299601:C:CT                    c.972dupA  p.V325fs   
CP069124.1:1160157:A:AT                    c.457dupA  p.I153fs   
CP069124.1:898050:C:CT                    c.1550dupA  p.K518fs   
CP069124.1:877318:T:TA                     c.441dupA  p.A148fs   
CP069124.1:873531:C:CA    

In [16]:
t.isa('outgroup_0810PADBR-1', isa_column='outbreak', kind='dataframe').features_summary(selection='unique').sort_values('Count', ascending=False)

Sequence  Position Deletion Insertion   Type  Count  \
Mutation                                                                        
CP069124.1:1078303:C:T  CP069124.1   1078303        C         T    SNP      8   
CP069124.1:208864:G:T   CP069124.1    208864        G         T    SNP      8   
CP069124.1:353600:A:AT  CP069124.1    353600        A        AT  INDEL      8   
CP069124.1:1161960:C:G  CP069124.1   1161960        C         G    SNP      8   
CP069124.1:206253:A:G   CP069124.1    206253        A         G    SNP      8   
...                            ...       ...      ...       ...    ...    ...   
CP069124.1:157818:A:T   CP069124.1    157818        A         T    SNP      1   
CP069124.1:299507:C:T   CP069124.1    299507        C         T    SNP      1   
CP069124.1:301188:T:G   CP069124.1    301188        T         G    SNP      1   
CP069124.1:301254:T:G   CP069124.1    301254        T         G    SNP      1   
CP069124.1:642470:T:C   CP069124.1    642470        T         C    SNP      1   

                        Total  Percent          Annotation Annotation_Impact  \
Mutation                                                                       
CP069124.1:1078303:C:T      8    100.0  synonymous_variant               LOW   
CP069124.1:208864:G:T       8    100.0    missense_variant          MODERATE   
CP069124.1:353600:A:AT      8    100.0      intron_variant          MODIFIER   
CP069124.1:1161960:C:G      8    100.0      intron_variant          MODIFIER   
CP069124.1:206253:A:G       8    100.0    missense_variant          MODERATE   
...                       ...      ...                 ...               ...   
CP069124.1:157818:A:T       8     12.5    missense_variant          MODERATE   
CP069124.1:299507:C:T       8     12.5  synonymous_variant               LOW   
CP069124.1:301188:T:G       8     12.5  synonymous_variant               LOW   
CP069124.1:301254:T:G       8     12.5  synonymous_variant               LOW   
CP069124.1:642470:T:C       8     12.5      intron_variant          MODIFIER   

                           Gene_Name       Gene_ID Feature_Type  \
Mutation                                                          
CP069124.1:1078303:C:T          cmeF  A0X86_005700   transcript   
CP069124.1:208864:G:T   A0X86_001120  A0X86_001120   transcript   
CP069124.1:353600:A:AT          hisA  A0X86_000005   transcript   
CP069124.1:1161960:C:G          hisA  A0X86_000005   transcript   
CP069124.1:206253:A:G   A0X86_001110  A0X86_001110   transcript   
...                              ...           ...          ...   
CP069124.1:157818:A:T   A0X86_000845  A0X86_000845   transcript   
CP069124.1:299507:C:T           fliI  A0X86_001640   transcript   
CP069124.1:301188:T:G   A0X86_001650  A0X86_001650   transcript   
CP069124.1:301254:T:G   A0X86_001650  A0X86_001650   transcript   
CP069124.1:642470:T:C           hisA  A0X86_000005   transcript   

                       Transcript_BioType                       HGVS.c  \
Mutation                                                                 
CP069124.1:1078303:C:T     protein_coding                    c.2034C>T   
CP069124.1:208864:G:T      protein_coding                     c.336G>T   
CP069124.1:353600:A:AT     protein_coding  c.350+353250_350+353251insT   
CP069124.1:1161960:C:G     protein_coding              c.351-472549C>G   
CP069124.1:206253:A:G      protein_coding                     c.541A>G   
...                                   ...                          ...   
CP069124.1:157818:A:T      protein_coding                    c.3607A>T   
CP069124.1:299507:C:T      protein_coding                    c.1362C>T   
CP069124.1:301188:T:G      protein_coding                     c.414A>C   
CP069124.1:301254:T:G      protein_coding                     c.348A>C   
CP069124.1:642470:T:C      protein_coding              c.350+642120T>C   

                          HGVS.p  \
Mutation                           
CP069124.1:1078303:C:T  